In [1]:
import random
import string
import time
import tensorflow as tf
import numpy as np
from scipy import sparse

In [284]:
list_of_strs = [''.join([string.ascii_lowercase[random.randrange(0, 26)] for i in range(random.randrange(4, 20))])
                for j in range(500)]

In [285]:
list_of_chars = set.union(*[set(i) for i in list_of_strs])

In [286]:
char_dict = {value:idx for idx, value in enumerate(sorted(list_of_chars))}

In [287]:
char_dict[' '] = max(char_dict.values())+1

In [288]:
max_length = max([len(i) for i in list_of_strs])

In [289]:
def onehot_string(x, max_length):
    
    if x!=x:
        return None
    
    #Pad to max length
    while len(x)<max_length:
        x+=' '
        
    sparse_matrix = sparse.hstack([sparse.csr_matrix([0 if char_dict[current_char]!=i else 1 for i in range(len(char_dict))]) for current_char in x])
    
    return sparse_matrix

In [290]:
n_tiles = 500 # dense_strs.shape[0]

In [ ]:
start_time = time.time()
sparse_strs = sparse.vstack([onehot_string(str_, max_length) for str_ in list_of_strs])
end_time = time.time()

In [ ]:
max_length = dense_strs.shape[1]

In [ ]:
dense_strs = sparse_strs.toarray()

In [ ]:
dense_strs.shape

In [ ]:
vector_data = tf.placeholder(dtype=tf.float32, shape=dense_strs.shape)

In [ ]:
tiled_vector_data = tf.reshape(tf.tile(vector_data, multiples=[1, n_tiles]), 
                               shape=[dense_strs.shape[0], n_tiles, dense_strs.shape[1]])

In [277]:
tile_ref_data = tf.reshape(tf.tile(vector_data, multiples=[n_tiles, 1]), 
                           shape=[dense_strs.shape[0], n_tiles, dense_strs.shape[1]])

In [278]:
not_equal = tf.cast(tf.equal(tile_ref_data, tiled_vector_data), dtype=tf.float32)

In [279]:
dist_sum = tf.reduce_sum(not_equal, axis=-1)

In [280]:
nearest_neighbors = tf.nn.top_k(dist_sum, k=2, sorted=True)

In [281]:
_, hamming_top_args = tf.nn.top_k(nearest_neighbors.values[:,1], k=nearest_neighbors.values.shape[0])

In [282]:
#nearest_neighbors.values[hamming_top_args]

In [283]:
with tf.Session() as sess:
    t = sess.run([hamming_top_args, nearest_neighbors.indices, nearest_neighbors.values], {vector_data:dense_strs})

ResourceExhaustedError: OOM when allocating tensor with shape[1000000,513]
	 [[Node: Tile_15 = Tile[T=DT_FLOAT, Tmultiples=DT_INT32, _device="/job:localhost/replica:0/task:0/gpu:0"](_arg_Placeholder_5_0_0/_1, Tile_15/multiples)]]

Caused by op 'Tile_15', defined at:
  File "C:\Users\euix\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\euix\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\euix\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\euix\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\euix\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\euix\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "C:\User

In [255]:
def map_nearest_neighbor(string_list, top_args, top_values):
    
    if type(string_list)!=np.array:
        string_list = np.array(string_list)
    
    results = []
    values = []
    for idx, top_arg in enumerate(top_args):
        results.append(string_list[top_arg])
        values.append(top_values[top_args])
    return np.array(results), np.array(values)

In [257]:
nn_results = map_nearest_neighbor(list_of_strs, t[1][t[0]], t[2][t[0]])

In [261]:
nn_results[0][0:2]

array([['mdnve', 'futre'],
       ['rnaq', 'djpx']], dtype='<U19')

In [262]:
nn_results[1][0:2]

array([[[[513., 505.],
         [513., 485.]],

        [[513., 499.],
         [513., 481.]],

        [[513., 485.],
         [513., 505.]],

        [[513., 481.],
         [513., 499.]],

        [[513., 505.],
         [513., 483.]],

        [[513., 503.],
         [513., 505.]],

        [[513., 503.],
         [513., 497.]],

        [[513., 503.],
         [513., 491.]],

        [[513., 501.],
         [513., 483.]],

        [[513., 497.],
         [513., 503.]],

        [[513., 495.],
         [513., 495.]],

        [[513., 495.],
         [513., 505.]],

        [[513., 495.],
         [513., 505.]],

        [[513., 491.],
         [513., 503.]],

        [[513., 483.],
         [513., 505.]],

        [[513., 483.],
         [513., 505.]],

        [[513., 503.],
         [513., 505.]],

        [[513., 503.],
         [513., 485.]],

        [[513., 503.],
         [513., 503.]],

        [[513., 503.],
         [513., 505.]],

        [[513., 501.],
         [513., 5